<a href="https://colab.research.google.com/github/Sushil211/CNN/blob/master/cnn_on_mnist_with_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! cp kaggle.json /root/.kaggle/kaggle.json

In [6]:
! kaggle

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: too few arguments


In [7]:
! chmod 600 /root/.kaggle/kaggle.json

In [8]:
! kaggle datasets download ellenyusa/4classimages

 99% 38.0M/38.5M [00:01<00:00, 14.3MB/s]
100% 38.5M/38.5M [00:01<00:00, 27.5MB/s]


In [9]:
! unzip -qq 4classimages.zip

In [10]:
! rm -rf data/data

In [14]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator

from tensorflow.keras.applications import ResNet50

In [15]:
gen = ImageDataGenerator(rotation_range=15, shear_range=0.1, zoom_range=0.3)

In [16]:
generator = gen.flow_from_directory("data/")

Found 808 images belonging to 4 classes.


In [17]:
resnet_model = ResNet50(include_top=False, weights='imagenet',input_shape=(256, 256, 3))

94773248/94765736 [==============================] - 1s 0us/step


In [18]:
for layer in resnet_model.layers:
  layer.trainable = False

In [19]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

Total params: 23,587,712<br>
Trainable params: 0<br>
Non-trainable params: 23,587,712

<b>Note how Trainable params = 0, i.e all the params are non-trainable</b>

In [20]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model



in_layer = Input(shape=(256, 256, 3))

resnet_layer = resnet_model(in_layer) #Here our resnet_model is working as a layer

flat = Flatten()(resnet_layer)
d1 = Dense(units=400, activation="tanh")(flat)
d2 = Dense(units=100, activation="tanh")(d1)
out_layer = Dense(units=4, activation="softmax")(d2)

In [21]:
model = Model(in_layer, out_layer)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet50 (Model)             (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 400)               52429200  
_________________________________________________________________
dense_1 (Dense)              (None, 100)               40100     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total params: 76,057,416
Trainable params: 52,469,704
Non-trainable params: 23,587,712
________________________________________

In [22]:
model.compile(optimizer= 'adam', loss = "categorical_crossentropy", metrics=['accuracy'])

In [23]:
model.fit_generator(generator, steps_per_epoch=20, epochs=5)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
20/20 [==============================] - 12s 592ms/step - loss: 0.7536 - accuracy: 0.7484
Epoch 2/5
20/20 [==============================] - 12s 600ms/step - loss: 0.2472 - accuracy: 0.9219
Epoch 3/5
20/20 [==============================] - 12s 593ms/step - loss: 0.1639 - accuracy: 0.9399
Epoch 4/5
20/20 [==============================] - 12s 606ms/step - loss: 0.1483 - accuracy: 0.9448
Epoch 5/5
20/20 [==============================] - 12s 603ms/step - loss: 0.1280 - accuracy: 0.9656


<b>Observe, how our accuracy increased to 96% by using resnet model, which was previously only 23% without using it.</b>

In [30]:
#Now testing our model from a random picture from internet
! wget https://media.gettyimages.com/photos/dreamhorse-picture-id157435265?s=612x612

#This is a horse image

--2020-06-24 05:37:31--  https://media.gettyimages.com/photos/dreamhorse-picture-id157435265?s=612x612
Resolving media.gettyimages.com (media.gettyimages.com)... 13.35.34.25, 13.35.34.31, 13.35.34.61, ...
Connecting to media.gettyimages.com (media.gettyimages.com)|13.35.34.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39531 (39K) [image/jpeg]
Saving to: ‘dreamhorse-picture-id157435265?s=612x612’

dreamhorse-picture- 100%[===================>]  38.60K  --.-KB/s    in 0.003s  

2020-06-24 05:37:32 (11.1 MB/s) - ‘dreamhorse-picture-id157435265?s=612x612’ saved [39531/39531]



In [31]:
img = np.array(load_img("dreamhorse-picture-id157435265?s=612x612", target_size=(256, 256)))

In [32]:
img.shape #Automatically converted to 3d

(256, 256, 3)

In [34]:
model.predict(np.array([img]))
np.argmax(model.predict(np.array([img])), axis = 1)

array([3])

In [29]:
generator.class_indices

{'Humans': 0, 'cats': 1, 'dogs': 2, 'horses': 3}

<b>So, this model is able to give us the correct result</b>

In [35]:
#Let's try on a dog image

! wget https://a57.foxnews.com/static.foxnews.com/foxnews.com/content/uploads/2019/04/931/524/white-pomeranian-dog-istock.jpg?ve=1&tl=1 

--2020-06-24 05:40:57--  https://a57.foxnews.com/static.foxnews.com/foxnews.com/content/uploads/2019/04/931/524/white-pomeranian-dog-istock.jpg?ve=1
Resolving a57.foxnews.com (a57.foxnews.com)... 23.73.24.27
Connecting to a57.foxnews.com (a57.foxnews.com)|23.73.24.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80172 (78K) [image/jpeg]
Saving to: ‘white-pomeranian-dog-istock.jpg?ve=1’

white-pomeranian-do 100%[===================>]  78.29K  --.-KB/s    in 0.07s   

2020-06-24 05:40:58 (1.17 MB/s) - ‘white-pomeranian-dog-istock.jpg?ve=1’ saved [80172/80172]



In [36]:
img = np.array(load_img("white-pomeranian-dog-istock.jpg?ve=1", target_size=(256, 256)))
model.predict(np.array([img]))
np.argmax(model.predict(np.array([img])), axis = 1)

array([2])

In [37]:
generator.class_indices

{'Humans': 0, 'cats': 1, 'dogs': 2, 'horses': 3}

<b>So also working for a dog image</b>